# Retrieve from XML Database and Count Key Words

In [1]:
import pyodbc
import numpy as np
import pandas as pd

SERVER_NAME = 'TSTWEOSQL,5876'
DATABASE_NAME = 'IMF_EPUBS'

QUERY_STRING = "SELECT dbo.PUBLICATION.*, dbo.DOCUMENT.* \
                FROM [dbo].[PUBLICATION] \
                INNER JOIN dbo.DOCUMENT ON dbo.PUBLICATION.SeriesNumber = dbo.DOCUMENT.SeriesNumber \
                WHERE dbo.PUBLICATION.ImfCategoryDesc = 'IMF Staff Country Reports' \
                AND dbo.PUBLICATION.ProjectedYear in(2017,2018)\
                AND dbo.PUBLICATION.Description like '%Article IV%'" 

searchList = ['FinTech','TechFin','digital','technology','Blockchain','innovation',
              'distributed ledger','Bitcoin','ICO','cryptocurrency','mobile','online','cyber','decentralization',
              'international payment','correspondent bank','financial inclusion','InsurTech','RegTech','micro lending',
              'machine learning','big data', 'artificial intelligence','analytics']

## Load XML files from IMF ODBC database using SQL query

In [2]:
def get_xmls(server_name, database_name, query_string):
    CON = pyodbc.connect('Trusted_Connection=yes',
                         driver='{SQL Server}',
                         server='%s' % server_name,
                         database='%s' % database_name)

    CURSOR = CON.cursor()
    return CURSOR.execute(query_string).fetchall()


results = get_xmls(SERVER_NAME, DATABASE_NAME, QUERY_STRING)

In [3]:
len(results)
# print(results[1].Content)

347

## Count frequency of words from the search List

In [4]:
def count_words(word, document):
    "return frequency of word appearing in document"
    return document.count(word)

v_count_words = np.vectorize(count_words)

In [5]:
count_all = [v_count_words(searchList, doc.Content) for doc in results]
#v_count_words(searchList, results[1].Content)
df = pd.DataFrame(data = count_all, columns = searchList)
output = df.sum()
output = pd.DataFrame(data = output, columns =['Freq'])
output.sort_values('Freq', ascending= False)

,Freq
innovation,153
financial inclusion,114
correspondent bank,104
mobile,63
technology,55
FinTech,28
online,23
decentralization,19
digital,18
analytics,10
